# Quantum Data Encoding Comparison

Comparing basis, angle, dense angle, and amplitude encoding strategies
in terms of circuit resources and classification performance.

In [ ]:
import numpy as np
import sys
sys.path.append('..')

from src.encoders import angle_encoding, dense_angle_encoding, amplitude_encode_data
from src.resource_analysis import compare_encoding_resources, resource_summary_table
from src.classification import build_vqc_with_encoding, train_vqc, evaluate_vqc
from src.data_utils import load_moons, load_iris_binary
from src.plotting import plot_resource_comparison, plot_accuracy_comparison, plot_training_curves

## 1. Circuit Resource Analysis

Compare qubits, depth, and gate count across encoding methods.

In [ ]:
n_features_range = [2, 4, 6, 8]
resources = compare_encoding_resources(n_features_range)
resource_summary_table(n_features_range, resources)
plot_resource_comparison(n_features_range, resources)

## 2. Classification with Different Encodings

Train VQCs using angle and dense angle encoding on the moons dataset.

In [ ]:
X_train, X_test, y_train, y_test = load_moons(n_samples=100, seed=42)
print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")
print(f"Features: {X_train.shape[1]}")

results = {}

for enc_type in ['angle', 'dense_angle']:
    print(f"\nTraining with {enc_type} encoding...")
    circuit, dp, tp = build_vqc_with_encoding(enc_type, n_features=2, n_layers=2)
    train_result = train_vqc(circuit, dp, tp, X_train, y_train,
                              maxiter=40, shots=512, seed=42)
    accuracy = evaluate_vqc(circuit, dp, tp, train_result['weights'],
                             X_test, y_test)
    results[enc_type] = train_result
    results[enc_type]['accuracy'] = accuracy
    print(f"  Test accuracy: {accuracy:.3f}")

In [ ]:
plot_training_curves(results)
plot_accuracy_comparison(
    list(results.keys()),
    [r['accuracy'] for r in results.values()],
    dataset_name='Moons'
)

## 3. Iris Dataset (4 features)

Test on a higher-dimensional dataset where qubit savings from
dense encoding become more significant.

In [ ]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = load_iris_binary(seed=42)
print(f"Iris features: {X_train_iris.shape[1]}")

iris_results = {}
for enc_type in ['angle', 'dense_angle']:
    print(f"\nTraining with {enc_type} encoding on Iris...")
    circuit, dp, tp = build_vqc_with_encoding(enc_type, n_features=4, n_layers=2)
    train_result = train_vqc(circuit, dp, tp, X_train_iris, y_train_iris,
                              maxiter=40, shots=512, seed=42)
    accuracy = evaluate_vqc(circuit, dp, tp, train_result['weights'],
                             X_test_iris, y_test_iris)
    iris_results[enc_type] = {'accuracy': accuracy}
    print(f"  Test accuracy: {accuracy:.3f}")

print("\nSummary:")
for name, r in iris_results.items():
    print(f"  {name}: {r['accuracy']:.3f}")